In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) Platinum 8280 CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)
Environment:
  JULIA_DEPOT_PATH = /work/go85/o85002/.julia
  JULIA_DIR = /work/opt/local/apps/julia/1.4.0


In [2]:
mycommand = `pwd`
run(mycommand)

/work/00/go85/o85002/non_eq_relax/J2_-5e-3/L300


Process(`pwd`, ProcessExited(0))

In [3]:
using Interpolations
using Printf
using Optim
using PyPlot
using LinearAlgebra
using HDF5

In [4]:
function read_from_hdf5(h5file,obs_name)
	fid = h5open(h5file,"r")
	temps = fid["temperatures"][:]
	obs   = fid[obs_name*"/mean"]
	times = [i for i in 1:size(obs)[1]]
	return temps,obs,times
end

read_from_hdf5 (generic function with 1 method)

In [5]:
h5file = "2d_out.h5"
obs_name = "afvc_corr"
temps,m_data,times = read_from_hdf5(h5file,obs_name)

([0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1], HDF5 dataset: /afvc_corr/mean (file: 2d_out.h5 xfer_mode: 0), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000, 10001])

In [6]:
# cut too low temperature datas
num_temps = length(temps)
min_temp  = 1
max_temp  = num_temps
for it in min_temp:max_temp
	plt.semilogy(times,m_data[:,it],label="$(temps[it])")
end
plt.title("$(obs_name) J2/J1 = -5e-3, L= 900")
plt.xscale("log")
plt.xlim(1,length(times)*15)
plt.ylim(1e-6,1)

plt.legend(loc="upper right")
plt.show()

PyCall.PyError: [91mPyError ($(Expr(:escape, :(ccall(#= /work/go85/o85002/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <type 'exceptions.ValueError'>[39m
[91mValueError('Data has no positive values, and therefore can not be log-scaled.',)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/pyplot.py", line 2954, in semilogy[39m
[91m    ret = ax.semilogy(*args, **kwargs)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 4232, in semilogy[39m
[91m    l = self.plot(*args, **kwargs)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 4000, in plot[39m
[91m    self.autoscale_view(scalex=scalex, scaley=scaley)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 1967, in autoscale_view[39m
[91m    y0, y1 = ylocator.view_limits(y0, y1)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/ticker.py"

In [7]:
#=
#load average data from text file
temps = [0.112,0.113,0.114,0.115,0.116]
file_idx = [3,4,5,6,7]
num_temps = length(temps)
mc_steps  = 20000 
m_data = Array{Float64}(undef,mc_steps,num_temps)
counter = 1
for ifl in file_idx
	open("mq_sqrt3_$(ifl).dat","r") do fp
		for is in 1:mc_steps
			str = split(readline(fp))
			m_data[is,counter] = parse(Float64,str[2])
		end
	end
	counter += 1
end
=#

In [8]:
m_data

HDF5 dataset: /afvc_corr/mean (file: 2d_out.h5 xfer_mode: 0)

In [9]:
num_temps = length(temps)
println(num_temps)
times = [i for i in 1:size(m_data)[1]]
for it in 1:num_temps
	plt.semilogy(times[10:end],m_data[:,it][10:end],label="$(temps[it])")
end
plt.title("$(obs_name) J2/J1 = -5e-3, L= 900")
#plt.xscale("log")
plt.xlim(1,length(times)*1.5)
plt.ylim(1e-3,1)
plt.legend(loc="upper right")
plt.show()

19


PyCall.PyError: [91mPyError ($(Expr(:escape, :(ccall(#= /work/go85/o85002/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <type 'exceptions.ValueError'>[39m
[91mValueError('Data has no positive values, and therefore can not be log-scaled.',)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/pyplot.py", line 2954, in semilogy[39m
[91m    ret = ax.semilogy(*args, **kwargs)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 4232, in semilogy[39m
[91m    l = self.plot(*args, **kwargs)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 4000, in plot[39m
[91m    self.autoscale_view(scalex=scalex, scaley=scaley)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 1967, in autoscale_view[39m
[91m    y0, y1 = ylocator.view_limits(y0, y1)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/ticker.py"

In [10]:
# reshape some datas for optimization.
min_temp = 1
max_temp = 5
temps  = temps[min_temp:max_temp]
num_temps = length(temps)
m_data = m_data[:,min_temp:max_temp]


10001×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 ⋮                   
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [11]:
# search configuration of tau which put some curves on one curve.
taus   = [120,95.,50.,30.,20.0]
@assert length(taus) == max_temp - min_temp + 1
lambda = 0.1
start  = 1000
plt.figure()
for it in 1:num_temps
	plt.semilogy(times[start:end] ./ taus[it],(taus[it]^lambda)*m_data[:,it][start:end],marker="o",label="$(temps[it])")
	
end
plt.title("scaling plot before τ optimization")
plt.xlabel("t/τ")
plt.ylabel("τ^λ * raw_data")
plt.legend(loc="upper right")
plt.show()

PyCall.PyError: [91mPyError ($(Expr(:escape, :(ccall(#= /work/go85/o85002/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <type 'exceptions.ValueError'>[39m
[91mValueError('Data has no positive values, and therefore can not be log-scaled.',)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/pyplot.py", line 2954, in semilogy[39m
[91m    ret = ax.semilogy(*args, **kwargs)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 4232, in semilogy[39m
[91m    l = self.plot(*args, **kwargs)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 4000, in plot[39m
[91m    self.autoscale_view(scalex=scalex, scaley=scaley)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/axes.py", line 1967, in autoscale_view[39m
[91m    y0, y1 = ylocator.view_limits(y0, y1)[39m
[91m  File "/usr/lib64/python2.7/site-packages/matplotlib/ticker.py"

In [12]:
"""
params: [tau1, tau2, ...]
"""
newaxis = [CartesianIndex()]
function cost(params)
    taus = params
    nx_smpl = 10
    x_u = minimum(times[end] ./ taus)
    x_l = maximum(times[start] ./ taus)
    xs_smpl = range(x_l, x_u, length=nx_smpl)
    g_intpl = zeros((nx_smpl, length(temps)))
    for it in eachindex(temps)
        tau = taus[it]
        xs = times ./ tau
        g_data = tau^lambda * m_data[:, it]
        itp = LinearInterpolation(xs, g_data)        
        g_intpl[:, it] = itp.(xs_smpl)
    end
    g_mean = sum(g_intpl, dims=2) ./ length(temps)
    norm(g_intpl .- g_mean[:, newaxis])^2 /  norm(g_mean)^2
end

cost (generic function with 1 method)

In [13]:
res = Optim.optimize(cost, taus)
res

BoundsError: [91mBoundsError: attempt to access 10001-element extrapolate(interpolate((::Array{Float64,1},), ::Array{Float64,1}, Gridded(Linear())), Throw()) with element type Float64 at index [100.25062656641603][39m

In [14]:
taus_opt = Optim.minimizer(res)

UndefVarError: [91mUndefVarError: res not defined[39m

In [15]:
println("T τ")
for i in 1:num_temps 
	println(temps[i]," ",taus_opt[i])
end


T τ


UndefVarError: [91mUndefVarError: taus_opt not defined[39m

In [16]:
plt.figure()
for it in eachindex(temps)
	plt.semilogy(times[10:end] ./ taus_opt[it], (taus_opt[it]^lambda) * m_data[:, it][10:end], marker="o",label="$(temps[it])")
    #println((taus[it]^lambda) * m_data[:, it])
end
plt.title("scaling plot after τ optimizaton")
plt.xlabel("t/τ")
plt.ylabel("τ^λ * m_data")
plt.legend(loc="upper right")
plt.show()

UndefVarError: [91mUndefVarError: taus_opt not defined[39m

In [17]:
x = 1.0
y = 0.9
n = 100
test = range(x,y,length=n)

1.0:-0.00101010101010101:0.9

In [18]:
test = LinRange(x,y,n)


100-element LinRange{Float64}:
 1.0,0.99899,0.99798,0.99697,0.99596,…,0.90404,0.90303,0.90202,0.90101,0.9